# PE for LISA BHB (10^6, 10^7)

In [48]:
from __future__ import division, print_function

import numpy as np

import bilby

outdir = 'outdir'
label = 'roqLISA'

In [49]:
# The ROQ bases can be given an overall scaling.
# Note how this is applied to durations, frequencies and masses.
scale_factor = 1.0

# Load in the pieces for the linear part of the ROQ. Note you will need to
# adjust the filenames here to the correct paths on your machine
basis_matrix_linear = np.load("B_linear.npy").T
freq_nodes_linear = np.load("fnodes_linear.npy") * scale_factor

# Load in the pieces for the quadratic part of the ROQ
basis_matrix_quadratic = np.load("B_quadratic.npy").T
freq_nodes_quadratic = np.load("fnodes_quadratic.npy") * scale_factor

# Load the parameters describing the valid parameters for the basis.
# params = np.genfromtxt("/Users/hxc/Downloads/ROQ_data-master/IMRPhenomPv2/4s/params.dat", names=True)
params = np.genfromtxt("params.dat", names=True)
params['flow'] *= scale_factor
params['fhigh'] *= scale_factor
params['seglen'] /= scale_factor
params['chirpmassmin'] /= scale_factor
params['chirpmassmax'] /= scale_factor
params['compmin'] /= scale_factor

In [61]:
# np.random.seed(170808)

duration = 1638400 / scale_factor   # deltaF = 1.0e-6
sampling_frequency = 2.0e-2 * scale_factor  # fmax = 1.0e-2

injection_parameters = dict(
    mass_1=5.0e6, mass_2=6.0e6, a_1=0.4, a_2=0.3, tilt_1=0.0, tilt_2=0.0,
    phi_12=1.7, phi_jl=0.3, luminosity_distance=1000., theta_jn=0.4, psi=0.659,
    phase=1.3, geocent_time=1126259642.413, ra=1.375, dec=-1.2108)

waveform_arguments = dict(waveform_approximant='IMRPhenomPv2',
                          reference_frequency=1.0e-4 * scale_factor,
                          minimum_frequency=1.0e-4 * scale_factor)

waveform_generator = bilby.gw.WaveformGenerator(
    duration=duration, sampling_frequency=sampling_frequency,
    frequency_domain_source_model=bilby.gw.source.lal_binary_black_hole,
    waveform_arguments=waveform_arguments,
    parameter_conversion=bilby.gw.conversion.convert_to_lal_binary_black_hole_parameters)

# print(len(waveform_generator.frequency_array))
# print(len(waveform_generator.time_array))

frequency_array = waveform_generator.frequency_array
ifos = bilby.gw.detector.InterferometerList(['H1','V1','L1'])
ifos.set_strain_data_from_zero_noise(
    sampling_frequency=sampling_frequency, duration=duration,
    start_time=injection_parameters['geocent_time'] - 1638400 +1)
ifos.inject_signal(waveform_generator=waveform_generator,
                   parameters=injection_parameters)


12:26 bilby INFO    : Injected signal in H1:
12:26 bilby INFO    :   optimal SNR = 0.00
12:26 bilby INFO    :   matched filter SNR = nan+nanj
12:26 bilby INFO    :   mass_1 = 5000000.0
12:26 bilby INFO    :   mass_2 = 6000000.0
12:26 bilby INFO    :   a_1 = 0.4
12:26 bilby INFO    :   a_2 = 0.3
12:26 bilby INFO    :   tilt_1 = 0.0
12:26 bilby INFO    :   tilt_2 = 0.0
12:26 bilby INFO    :   phi_12 = 1.7
12:26 bilby INFO    :   phi_jl = 0.3
12:26 bilby INFO    :   luminosity_distance = 1000.0
12:26 bilby INFO    :   theta_jn = 0.4
12:26 bilby INFO    :   psi = 0.659
12:26 bilby INFO    :   phase = 1.3
12:26 bilby INFO    :   geocent_time = 1126259642.413
12:26 bilby INFO    :   ra = 1.375
12:26 bilby INFO    :   dec = -1.2108
12:26 bilby INFO    : Injected signal in V1:
12:26 bilby INFO    :   optimal SNR = 0.00
12:26 bilby INFO    :   matched filter SNR = nan+nanj
12:26 bilby INFO    :   mass_1 = 5000000.0
12:26 bilby INFO    :   mass_2 = 6000000.0
12:26 bilby INFO    :   a_1 = 0.4
12:

16385
32768


[{'plus': array([0.-0.j, 0.-0.j, 0.-0.j, ..., 0.-0.j, 0.-0.j, 0.-0.j]),
  'cross': array([0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j])},
 {'plus': array([0.-0.j, 0.-0.j, 0.-0.j, ..., 0.-0.j, 0.-0.j, 0.-0.j]),
  'cross': array([0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j])},
 {'plus': array([0.-0.j, 0.-0.j, 0.-0.j, ..., 0.-0.j, 0.-0.j, 0.-0.j]),
  'cross': array([0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j])}]

In [62]:
# make ROQ waveform generator
search_waveform_generator = bilby.gw.waveform_generator.WaveformGenerator(
    duration=duration, sampling_frequency=sampling_frequency,
    frequency_domain_source_model=bilby.gw.source.binary_black_hole_roq,
    waveform_arguments=dict(
        frequency_nodes_linear=freq_nodes_linear,
        frequency_nodes_quadratic=freq_nodes_quadratic,
        reference_frequency=20. * scale_factor, waveform_approximant='IMRPhenomPv2'),
    parameter_conversion=bilby.gw.conversion.convert_to_lal_binary_black_hole_parameters)

# Here we add constraints on chirp mass and mass ratio to the prior
# priors = bilby.gw.prior.BBHPriorDict()

for key in ['a_1', 'a_2', 'tilt_1', 'tilt_2', 'theta_jn', 'phase', 'psi', 'ra',
            'dec', 'phi_12', 'phi_jl', 'luminosity_distance']:
    priors[key] = injection_parameters[key]
for key in ['mass_1', 'mass_2']:
    priors[key].minimum = max(priors[key].minimum, params['compmin'])
priors['chirp_mass'] = bilby.core.prior.Constraint(
    name='chirp_mass', minimum=float(params['chirpmassmin']),
    maximum=float(params['chirpmassmax']))
priors['mass_ratio'] = bilby.core.prior.Constraint(0.125, 1, name='mass_ratio')
priors['geocent_time'] = bilby.core.prior.Uniform(
    injection_parameters['geocent_time'] - 0.1,
    injection_parameters['geocent_time'] + 0.1, latex_label='$t_c$', unit='s')

In [63]:
likelihood = bilby.gw.likelihood.ROQGravitationalWaveTransient(
    interferometers=ifos, waveform_generator=search_waveform_generator,
    linear_matrix=basis_matrix_linear, quadratic_matrix=basis_matrix_quadratic,
    priors=priors, roq_params=params)

# write the weights to file so they can be loaded multiple times
likelihood.save_weights('weights.json')

# remove the basis matrices as these are big for longer bases
del basis_matrix_linear, basis_matrix_quadratic

# load the weights from the file
likelihood = bilby.gw.likelihood.ROQGravitationalWaveTransient(
    interferometers=ifos, waveform_generator=search_waveform_generator,
    weights='weights.json.npz', priors=priors)

result = bilby.run_sampler(
    likelihood=likelihood, priors=priors, sampler='pymultinest', npoints=500,
    injection_parameters=injection_parameters, outdir=outdir, label=label)

# Make a corner plot.
result.plot_corner()

16:26 bilby WARNING : The waveform_generator start_time is not equal to that of the provided interferometers. Overwriting the waveform_generator.


IndexError: index -1 is out of bounds for axis 0 with size 0